In [275]:
import pandas as pd
import numpy as np
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
import sklearn.metrics as mt
from sklearn.model_selection import cross_val_score, cross_validate
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 15685796541254037398
xla_global_id: -1
]


In [276]:
df = pd.read_csv("C:/Users/Park Yoonseo/OneDrive/문서/LG Aimers 강의자료/train.csv", encoding_errors='ignore') # 학습용 데이터
df_test = pd.read_csv("C:/Users/Park Yoonseo/OneDrive/문서/LG Aimers 강의자료/submission.csv", encoding_errors='ignore') # 테스트 데이터(제출파일의 데이터)
df = df.drop(['customer_idx', 'product_subcategory', 'product_modelname', 'lead_owner', 'business_subarea',
                          'customer_country.1'], axis = 1)
df

,bant_submit,customer_country,business_unit,com_reg_ver_win_rate,customer_type,enterprise,historical_existing_cnt,id_strategic_ver,it_strategic_ver,idit_strategic_ver,...,product_category,customer_position,response_corporate,expected_timeline,ver_cus,ver_pro,ver_win_rate_x,ver_win_ratio_per_bu,business_area,is_converted
0,1.00,/Quezon City/Philippines,AS,0.066667,End-Customer,Enterprise,NaN,NaN,NaN,NaN,...,multi-split,entry level,LGEPH,less than 3 months,1,0,0.003079,0.026846,corporate / office,True
1,1.00,/PH-00/Philippines,AS,0.066667,End-Customer,Enterprise,12.0,NaN,NaN,NaN,...,multi-split,ceo/founder,LGEPH,less than 3 months,1,0,0.003079,0.026846,corporate / office,True
2,1.00,/Kolkata /India,AS,0.088889,End-Customer,Enterprise,144.0,NaN,NaN,NaN,...,single-split,partner,LGEIL,less than 3 months,1,0,0.003079,0.026846,corporate / office,True
3,1.00,/Bhubaneswar/India,AS,0.088889,End-Customer,Enterprise,NaN,NaN,NaN,NaN,...,vrf,ceo/founder,LGEIL,less than 3 months,1,0,0.003079,0.026846,corporate / office,True
4,1.00,/Hyderabad/India,AS,0.088889,Specifier/ Influencer,Enterprise,NaN,NaN,NaN,NaN,...,multi-split,partner,LGEIL,less than 3 months,0,0,0.003079,0.026846,corporate / office,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59294,1.00,/Sląskie/Poland,AS,NaN,End Customer,SMB,NaN,NaN,NaN,NaN,...,vrf,entry level,LGEPL,3 months ~ 6 months,0,0,0.000026,0.028777,public facility,False
59295,0.75,/Bogotá DC /Colombia,AS,0.040000,Specifier/ Influencer,Enterprise,NaN,NaN,NaN,NaN,...,etc.,trainee,LGECB,9 months ~ 1 year,0,0,0.000026,0.028777,public facility,False
59296,0.75,/Pisco/Peru,AS,0.040000,Specifier/ Influencer,Enterprise,NaN,NaN,NaN,NaN,...,single-split,entry level,LGEPR,less than 3 months,0,0,0.000026,0.028777,public facility,False
59297,1.00,/santa cruz bolivia/Peru,AS,0.040000,NaN,Enterprise,NaN,NaN,NaN,NaN,...,etc.,others,LGEPR,more than a year,0,0,0.000026,0.028777,public facility,False


In [277]:
df_test = df_test.drop(['bant_submit', 'com_reg_ver_win_rate', 'historical_existing_cnt', 'id_strategic_ver',
                          'it_strategic_ver', 'ver_cus', 'ver_pro', 'ver_win_rate_x', 'ver_win_ratio_per_bu'], axis = 1)
df_test.head()

,id,customer_country,business_unit,customer_idx,customer_type,enterprise,idit_strategic_ver,customer_job,lead_desc_length,inquiry_type,...,product_subcategory,product_modelname,customer_country.1,customer_position,response_corporate,expected_timeline,business_area,business_subarea,lead_owner,is_converted
0,19844,/ / Brazil,ID,47466,End Customer,Enterprise,NaN,consulting,3,NaN,...,NaN,NaN,/ / Brazil,none,LGESP,NaN,retail,Electronics & Telco,278,True
1,9738,400 N State Of Franklin Rd Cloud IT / Johnson...,IT,5405,End Customer,SMB,NaN,NaN,23,Quotation or Purchase Consultation,...,NaN,NaN,400 N State Of Franklin Rd Cloud IT / Johnson...,none,LGEUS,NaN,transportation,Others,437,True
2,8491,/ / U.A.E,ID,13597,Specifier/ Influencer,SMB,NaN,information technology,138,Quotation or Purchase Consultation,...,UT662M Series,50UT662M (MEA),/ / U.A.E,manager,LGEGF,less than 3 months,hospital & health care,General Hospital,874,False
3,19895,/ Madison / United States,ID,17204,NaN,Enterprise,NaN,sales,3,Quotation or Purchase Consultation,...,Digital Signage or Commercial TVs,NaN,/ Madison / United States,none,LGEUS,more than a year,retail,NaN,194,False
4,10465,/ Sao Paulo / Brazil,ID,2329,End Customer,Enterprise,1.0,engineering,107,Quotation or Purchase Consultation,...,LG LED Bloc,LSAA012,/ Sao Paulo / Brazil,others,LGESP,less than 3 months,corporate / office,Engineering,167,False


In [278]:
print(df.shape)
print(df_test.shape)
df_test.head()

(59299, 23)
(5271, 21)


,id,customer_country,business_unit,customer_idx,customer_type,enterprise,idit_strategic_ver,customer_job,lead_desc_length,inquiry_type,...,product_subcategory,product_modelname,customer_country.1,customer_position,response_corporate,expected_timeline,business_area,business_subarea,lead_owner,is_converted
0,19844,/ / Brazil,ID,47466,End Customer,Enterprise,NaN,consulting,3,NaN,...,NaN,NaN,/ / Brazil,none,LGESP,NaN,retail,Electronics & Telco,278,True
1,9738,400 N State Of Franklin Rd Cloud IT / Johnson...,IT,5405,End Customer,SMB,NaN,NaN,23,Quotation or Purchase Consultation,...,NaN,NaN,400 N State Of Franklin Rd Cloud IT / Johnson...,none,LGEUS,NaN,transportation,Others,437,True
2,8491,/ / U.A.E,ID,13597,Specifier/ Influencer,SMB,NaN,information technology,138,Quotation or Purchase Consultation,...,UT662M Series,50UT662M (MEA),/ / U.A.E,manager,LGEGF,less than 3 months,hospital & health care,General Hospital,874,False
3,19895,/ Madison / United States,ID,17204,NaN,Enterprise,NaN,sales,3,Quotation or Purchase Consultation,...,Digital Signage or Commercial TVs,NaN,/ Madison / United States,none,LGEUS,more than a year,retail,NaN,194,False
4,10465,/ Sao Paulo / Brazil,ID,2329,End Customer,Enterprise,1.0,engineering,107,Quotation or Purchase Consultation,...,LG LED Bloc,LSAA012,/ Sao Paulo / Brazil,others,LGESP,less than 3 months,corporate / office,Engineering,167,False


In [279]:
print(df['customer_country'].unique()[:10])
print(df['customer_country'].unique()[-10:])

['/Quezon City/Philippines' '/PH-00/Philippines' '/Kolkata /India'
 '/Bhubaneswar/India' '/Hyderabad/India' '/Abuja/Nigeria'
 '/Jeddah, KSA/Saudi Arabia' '/Guwahati/India' '/Cebu city/Philippines'
 '/hauz khas,delhi/India']
['/Itaú de Minas - Minas Gerais/Brazil' '/SP/ Jundiai/Brazil'
 '/Sergipe/Aracaju/Brazil' '/Estado de Mexico /Mexico'
 '/Baturaja/Indonesia' '/Sląskie/Poland' '/Bogotá DC /Colombia'
 '/Pisco/Peru' '/santa cruz bolivia/Peru' '/paris/France']


In [280]:
print(df_test['customer_country'].unique()[:10])
print(df_test['customer_country'].unique()[-10:])

['/  / Brazil'
 '400 N State Of Franklin Rd  Cloud IT / Johnson City / United States'
 '/  / U.A.E' '/ Madison  / United States' '/ Sao Paulo / Brazil'
 '/  / Germany' '/  / Egypt' '/  / France' '/  /'
 'Education /  / UNITED STATES']
['10990 ROE AVE   / KS / United States' '/ shihor / India'
 '/ Amarillo / United States' '/ Hurgada / Egypt' '/ Hajipur / India'
 '/ test / Greece' '/ Baradwar  / India' '/Sandur /India'
 '/ OURO BRANCO / Brazil' '/ Ongole  / India']


In [281]:
df_test['customer_country'] = df_test['customer_country'].apply(lambda x: x.split('/')[-1].strip())

# Printing the unique values to verify
print(df_test['customer_country'].unique())

['Brazil' 'United States' 'U.A.E' 'Germany' 'Egypt' 'France' ''
 'UNITED STATES' 'Indonesia' 'India' 'Italy' 'Argentina' 'Türkiye'
 'Saudi Arabia' 'Philippines' 'Peru' 'Vietnam' 'Australia' 'Chile'
 'Colombia' 'Hong Kong' 'Mexico' 'Thailand' 'Serbia'
 'Bosnia and Herzegovina' 'Netherlands' 'Hungary' 'Panama'
 'United Kingdom' 'South Africa' 'Maldives' 'Singapore' 'Ecuador'
 'Venezuela' 'NE' 'Israel' 'Austria' 'Guatemala' 'Belgium' 'Oman'
 'Tunisia' 'Nicaragua' 'Somalia' 'China' 'Canada'
 'Democratic Republic of the Congo' 'Poland' 'Dominican Republic' 'Spain'
 'Greece' 'Qatar' 'Trinidad and Tobago'
 '5003 Lady Of The Lake Dr  Raleigh NC' 'Kosovo' 'Algeria' 'Jordan'
 'Bulgaria' 'Cambodia' 'Romania' 'Georgia' 'El Salvador' 'Jamaica'
 'Englewood, CO' 'Honduras' 'Switzerland' 'Cyprus' 'Bangladesh' 'FL' 'TX'
 'Kuwait' 'Turks and Caicos Islands' 'Portugal' 'Uruguay' 'Costa Rica'
 'Puerto Rico' 'CA' 'Nigeria' 'St Maarten' 'Liberia' 'นนทบุร?Thailand'
 'Burkina Faso' 'Malaysia' 'Kenya' 'สมุทรปร

In [282]:
df_test.head()

,id,customer_country,business_unit,customer_idx,customer_type,enterprise,idit_strategic_ver,customer_job,lead_desc_length,inquiry_type,...,product_subcategory,product_modelname,customer_country.1,customer_position,response_corporate,expected_timeline,business_area,business_subarea,lead_owner,is_converted
0,19844,Brazil,ID,47466,End Customer,Enterprise,NaN,consulting,3,NaN,...,NaN,NaN,/ / Brazil,none,LGESP,NaN,retail,Electronics & Telco,278,True
1,9738,United States,IT,5405,End Customer,SMB,NaN,NaN,23,Quotation or Purchase Consultation,...,NaN,NaN,400 N State Of Franklin Rd Cloud IT / Johnson...,none,LGEUS,NaN,transportation,Others,437,True
2,8491,U.A.E,ID,13597,Specifier/ Influencer,SMB,NaN,information technology,138,Quotation or Purchase Consultation,...,UT662M Series,50UT662M (MEA),/ / U.A.E,manager,LGEGF,less than 3 months,hospital & health care,General Hospital,874,False
3,19895,United States,ID,17204,NaN,Enterprise,NaN,sales,3,Quotation or Purchase Consultation,...,Digital Signage or Commercial TVs,NaN,/ Madison / United States,none,LGEUS,more than a year,retail,NaN,194,False
4,10465,Brazil,ID,2329,End Customer,Enterprise,1.0,engineering,107,Quotation or Purchase Consultation,...,LG LED Bloc,LSAA012,/ Sao Paulo / Brazil,others,LGESP,less than 3 months,corporate / office,Engineering,167,False


In [283]:
print(df['business_unit'].nunique())
print(df['business_unit'].unique())

5
['AS' 'ID' 'IT' 'Solution' 'CM']


In [284]:
print(df_test['business_unit'].nunique())
print(df_test['business_unit'].unique())

3
['ID' 'IT' 'AS']


In [285]:
# 소문자 치환
df['customer_type'] = df['customer_type'].str.lower()
# other, others, etc를 other로 통합
df.loc[df['customer_type'].str.contains('other') == True,'customer_type'] = 'other'
df.loc[df['customer_type'].str.contains('etc') == True,'customer_type'] = 'other'
# end-customer, end-usr, end customer, commercial end-user을 end-customer로 치환
df.loc[df['customer_type'].str.contains('end') == True,'customer_type'] = 'end-customer'
print(df['customer_type'].nunique())
print(df['customer_type'].unique())

28
['end-customer' 'specifier/ influencer' 'service partner'
 'channel partner' nan 'corporate' 'specifier / influencer'
 'solution eco-partner' 'distributor' 'other' 'system integrator'
 'consultant' 'installer' 'homeowner' 'technician' 'installer/contractor'
 'engineer' 'manager / director' 'developer' 'architect/consultant'
 'hvac engineer' 'reseller' 'software/solution provider'
 'software / solution provider' 'dealer/distributor' 'technical assistant'
 'interior designer' 'home owner' 'administrator']


In [286]:
df_test['customer_type'] = df_test['customer_type'].str.lower()
# other, others, etc를 other로 통합
df_test.loc[df_test['customer_type'].str.contains('other') == True,'customer_type'] = 'other'
df_test.loc[df_test['customer_type'].str.contains('etc') == True,'customer_type'] = 'other'
# end-customer, end-usr, end customer, commercial end-user을 end-customer로 치환
df_test.loc[df_test['customer_type'].str.contains('end') == True,'customer_type'] = 'end-customer'
print(df_test['customer_type'].nunique())
print(df_test['customer_type'].unique())

6
['end-customer' 'specifier/ influencer' nan 'service partner'
 'channel partner' 'solution eco-partner' 'developer']


In [287]:
print(df['enterprise'].nunique())
print(df['enterprise'].unique())

2
['Enterprise' 'SMB']


In [288]:
print(df_test['enterprise'].nunique())
print(df_test['enterprise'].unique())

2
['Enterprise' 'SMB']


In [289]:
print(df['product_category'].count())
print(df['product_category'].nunique())
grouped_df = df.groupby('product_category')['is_converted'].count().reset_index()
grouped_df = grouped_df.sort_values(by='is_converted', ascending=False)
print(grouped_df[-30:])

39925
366
                                      product_category  is_converted
194                           multi-split,single-split             1
185                              monitorindustrial_rmk             1
163  monitor signage,commercial tv,monior/monitor t...             1
144  meeting & screen sharedirect view leddirect vi...             1
145                                 mobile,audio/video             1
146                      monior/monitor tv,audio/video             1
147                          monior/monitor tv,chiller             1
149            monior/monitor tv,projector,audio/video             1
150                     monior/monitor tv,refrigerator             1
151         monior/monitor tv,refrigerator,audio/video             1
152  monior/monitor tv,system ac,tv,pc,refrigerator...             1
153  monior/monitor tv,system ac,tv,refrigerator,wa...             1
155  monior/monitor tv,tv,commercial tv,pc,refriger...             1
160          monitor sig

In [290]:
print(df_test['product_category'].count())
print(df_test['product_category'].nunique())
grouped_df_test = df_test.groupby('product_category')['is_converted'].count().reset_index()
grouped_df_test = grouped_df_test.sort_values(by='is_converted', ascending=False)
print(grouped_df_test[-30:])

3413
33
             product_category  is_converted
13                led signage           289
5                        etc.           202
25               single-split           179
30         video wall signage           167
9                    hotel tv           134
28           standard signage           118
18               oled signage           104
1                     chiller            99
10  interactive digital board            74
6                     heating            67
15                    monitor            58
7     high brightness signage            40
14            medical display            32
29                ventilation            31
19           one:quick series            22
2                cloud device            18
32                      webos            14
12                     laptop            13
3                     control            13
23                  projector            12
26          software solution            11
22                pro:ce

In [291]:
# 소문자로 치환
df['product_category'] = df['product_category'].str.lower()

# null값 n으로 치환
df.loc[df['product_category'].isnull() == True, 'product_category'] = 'none'

# 기호들을 공백으로 치환
def replace_symbols(text):
    # 만약 문자열이 아니면 그대로 반환
    if not isinstance(text, str):
        return text
    return text.replace(',', ' ').replace('-', ' ').replace('/', ' ')

replaced_df = df['product_category'].apply(replace_symbols)
df['product_category'] = replaced_df
print(df['product_category'].count())
print(df['product_category'].nunique())
print(df['product_category'].unique())

59299
365
['multi split' 'single split' 'vrf' 'chiller' 'etc.' 'rac'
 'teto ou cassete inverter' 'none' 'software solution'
 'all lg vrf systems' 'ventilation' 'a thermodynamic water heater'
 'residential air conditioner' 'outros' 'heating' 'multi v5 vrf'
 'split tunggal' 'multi inverter' '?????' 'standard signage'
 'high brightness signage' 'interactive signage' 'video wall signage'
 'led signage' 'hotel tv' 'signage care solution' 'oled signage' 'idb'
 'video wall' 'one:quick series' 'commercial tv' 'lg one:quick'
 'special signage' 'other' 'accessories' 'commercial display'
 'interactive digital board' 'standard' 'uhd signage' 'smart tv signage'
 'lg one:quick series' 'pro:centric' 'ur640' 'ur640s' 'webos'
 'signage care solutions' 'aio | one quick' 'high brightness'
 'hospital tv' 'digital signage' 'monitor' 'pc' 'laptop' 'projector'
 'cloud device' 'medical display' 'washing machine dryer' 'solar chiller'
 'system ac rac' 'monitor signage commercial tv monior monitor tv'
 'monitor

In [292]:
# 소문자로 치환
df_test['product_category'] = df_test['product_category'].str.lower()

# null값 n으로 치환
df_test.loc[df_test['product_category'].isnull() == True, 'product_category'] = 'none'

# 기호들을 공백으로 치환
def replace_symbols(text):
    # 만약 문자열이 아니면 그대로 반환
    if not isinstance(text, str):
        return text
    return text.replace(',', ' ').replace('-', ' ').replace('/', ' ')

replaced_df_test = df_test['product_category'].apply(replace_symbols)
df_test['product_category'] = replaced_df_test
print(df_test['product_category'].count())
print(df_test['product_category'].nunique())
print(df_test['product_category'].unique())

5271
34
['none' 'hospital tv' 'led signage' 'cloud device' 'standard signage'
 'interactive signage' 'multi split' 'video wall signage' 'vrf' 'monitor'
 'etc.' 'heating' 'ventilation' 'control' 'single split'
 'high brightness signage' 'chiller' 'oled signage' 'special signage'
 'hotel tv' 'projector' 'medical display' 'signage care solution' 'ess'
 'laptop' 'interactive digital board' 'pro:centric' 'software solution'
 'others' 'one:quick series' 'webos' 'notebook' 'accessories' 'other']


In [293]:
# other, others, etc.를 other로 통합
df.loc[df['product_category'].str.contains('other') == True,'customer_type'] = 'other'
df.loc[df['product_category'].str.contains('etc') == True,'customer_type'] = 'other'

print(df['product_category'].count())
print(df['product_category'].nunique())
print(df['product_category'].unique()) 

59299
365
['multi split' 'single split' 'vrf' 'chiller' 'etc.' 'rac'
 'teto ou cassete inverter' 'none' 'software solution'
 'all lg vrf systems' 'ventilation' 'a thermodynamic water heater'
 'residential air conditioner' 'outros' 'heating' 'multi v5 vrf'
 'split tunggal' 'multi inverter' '?????' 'standard signage'
 'high brightness signage' 'interactive signage' 'video wall signage'
 'led signage' 'hotel tv' 'signage care solution' 'oled signage' 'idb'
 'video wall' 'one:quick series' 'commercial tv' 'lg one:quick'
 'special signage' 'other' 'accessories' 'commercial display'
 'interactive digital board' 'standard' 'uhd signage' 'smart tv signage'
 'lg one:quick series' 'pro:centric' 'ur640' 'ur640s' 'webos'
 'signage care solutions' 'aio | one quick' 'high brightness'
 'hospital tv' 'digital signage' 'monitor' 'pc' 'laptop' 'projector'
 'cloud device' 'medical display' 'washing machine dryer' 'solar chiller'
 'system ac rac' 'monitor signage commercial tv monior monitor tv'
 'monitor

In [294]:
df_test.loc[df_test['product_category'].str.contains('other') == True,'customer_type'] = 'other'
df_test.loc[df_test['product_category'].str.contains('etc') == True,'customer_type'] = 'other'

print(df_test['product_category'].count())
print(df_test['product_category'].nunique())
print(df_test['product_category'].unique())

5271
34
['none' 'hospital tv' 'led signage' 'cloud device' 'standard signage'
 'interactive signage' 'multi split' 'video wall signage' 'vrf' 'monitor'
 'etc.' 'heating' 'ventilation' 'control' 'single split'
 'high brightness signage' 'chiller' 'oled signage' 'special signage'
 'hotel tv' 'projector' 'medical display' 'signage care solution' 'ess'
 'laptop' 'interactive digital board' 'pro:centric' 'software solution'
 'others' 'one:quick series' 'webos' 'notebook' 'accessories' 'other']


In [295]:
# 그룹으로 묶어 카테고리별 데이터 수 확인
grouped_df = df.groupby('product_category')['is_converted'].count().reset_index()
grouped_df = grouped_df.sort_values(by='is_converted', ascending=False)
grouped_df[:30]

,product_category,is_converted
194,none,19374
108,interactive digital board,6154
330,vrf,5640
187,multi split,3676
325,video wall signage,2029
90,etc.,2018
128,led signage,1895
109,interactive signage,1862
250,single split,1574
228,rac,1447


In [296]:
# 그룹으로 묶어 카테고리별 데이터 수 확인
grouped_df_test = df_test.groupby('product_category')['is_converted'].count().reset_index()
grouped_df_test = grouped_df_test.sort_values(by='is_converted', ascending=False)
grouped_df_test[:30]

,product_category,is_converted
17,none,1858
32,vrf,726
16,multi split,544
11,interactive signage,400
13,led signage,289
5,etc.,202
26,single split,179
31,video wall signage,167
9,hotel tv,134
29,standard signage,118


In [297]:
# 그룹으로 묶어 카테고리별 데이터 수 확인
grouped_df = df.groupby('product_category')['is_converted'].count().reset_index()
grouped_df = grouped_df.sort_values(by='is_converted', ascending=False)
grouped_df[:30]

,product_category,is_converted
194,none,19374
108,interactive digital board,6154
330,vrf,5640
187,multi split,3676
325,video wall signage,2029
90,etc.,2018
128,led signage,1895
109,interactive signage,1862
250,single split,1574
228,rac,1447


In [298]:
# 그룹으로 묶어 카테고리별 데이터 수 확인
grouped_df_test = df_test.groupby('product_category')['is_converted'].count().reset_index()
grouped_df_test = grouped_df_test.sort_values(by='is_converted', ascending=False)
grouped_df_test[:30]

,product_category,is_converted
17,none,1858
32,vrf,726
16,multi split,544
11,interactive signage,400
13,led signage,289
5,etc.,202
26,single split,179
31,video wall signage,167
9,hotel tv,134
29,standard signage,118


In [299]:
# 카테고리가 20개 이하인 카테고리 none으로 치환
# 모두 합하면 854개(약 2.1%)
grouped_df = df.groupby('product_category')['is_converted'].count().reset_index()
grouped_df = grouped_df.sort_values(by='is_converted', ascending=False)
print(grouped_df[grouped_df['is_converted'] <= 20].sum())
grouped_df[grouped_df['is_converted'] <= 20]

product_category    servicesmonitor & pccommercial tv tvaire acond...
is_converted                                                      854
dtype: object


,product_category,is_converted
244,services,20
157,monitor & pc,19
74,commercial tv tv,19
45,aire acondicionado residencial,18
208,onequick series,16
...,...,...
176,monitor signage monior monitor tv system ac va...,1
177,monitor signage monior monitor tv tv audio video,1
178,monitor signage monior monitor tv vacuum clean...,1
180,monitor signage solar,1


In [300]:
# 카테고리가 20개 이하인 카테고리 none으로 치환
# 모두 합하면 127개(약 2.1%)
grouped_df_test = df_test.groupby('product_category')['is_converted'].count().reset_index()
grouped_df_test = grouped_df_test.sort_values(by='is_converted', ascending=False)
print(grouped_df_test[grouped_df_test['is_converted'] <= 20].sum())
grouped_df_test[grouped_df_test['is_converted'] <= 20]

product_category    cloud deviceweboslaptopcontrolprojectorsoftwar...
is_converted                                                      127
dtype: object


,product_category,is_converted
2,cloud device,18
33,webos,14
12,laptop,13
3,control,13
24,projector,12
27,software solution,11
23,pro:centric,10
8,hospital tv,10
28,special signage,9
0,accessories,6


In [301]:
def replace_words_with_n(text, word_list):
    if text in word_list.values:
        return 'none'
    else:
        return text

In [302]:
low_num_category = grouped_df_test[grouped_df_test['is_converted'] <= 20]['product_category']

# low_num_에 있는 단어를 'none'으로 치환하는 함수
# 데이터프레임의 'product_category' 열에 함수를 적용하여 치환
df_test['product_category'] = df_test['product_category'].apply(lambda x: replace_words_with_n(x, low_num_category))

print(df_test['product_category'].count())
print(df_test['product_category'].nunique())
print(df_test['product_category'].unique())

5271
19
['none' 'led signage' 'standard signage' 'interactive signage'
 'multi split' 'video wall signage' 'vrf' 'monitor' 'etc.' 'heating'
 'ventilation' 'single split' 'high brightness signage' 'chiller'
 'oled signage' 'hotel tv' 'medical display' 'interactive digital board'
 'one:quick series']


In [303]:
print(df['response_corporate'].nunique())
print(df['response_corporate'].unique())
grouped_df = df.groupby('response_corporate')['is_converted'].count().reset_index()
grouped_df = grouped_df.sort_values(by='is_converted', ascending=False)
grouped_df

53
['LGEPH' 'LGEIL' 'LGEAF' 'LGESJ' 'LGESL' 'LGESP' 'LGEGF' 'LGESA' 'LGEUS'
 'LGECB' 'LGEMS' 'LGEEG' 'LGEEF' 'LGEAP' 'LGEIN' 'LGEUK' 'LGEKR' 'LGEPS'
 'LGECI' 'LGECL' 'LGETK' 'LGELF' 'LGEPT' 'LGEPR' 'LGEDG' 'LGERO' 'LGEMK'
 'LGEPL' 'LGECZ' 'LGEES' 'LGEAR' 'LGEHK' 'LGEML' 'LGEJP' 'LGEHS' 'LGEAS'
 'LGEYK' 'LGEIS' 'LGEBN' 'LGEFS' 'LGESW' 'LGEMC' 'LGEAG' 'LGEEB' 'LGETH'
 'LGEVH' 'LGECH' 'LGELA' 'LGETT' 'LGERA' 'LGEUR' 'LGEIR' 'LGEBT']


,response_corporate,is_converted
21,LGEIL,16908
43,LGESP,9311
50,LGEUS,5955
32,LGEMS,2768
33,LGEPH,2651
18,LGEGF,2149
7,LGECB,2079
48,LGEUK,1651
41,LGESJ,1469
10,LGECL,1339


In [304]:
print(df_test['response_corporate'].nunique())
print(df_test['response_corporate'].unique())
grouped_df_test = df_test.groupby('response_corporate')['is_converted'].count().reset_index()
grouped_df_test = grouped_df_test.sort_values(by='is_converted', ascending=False)
grouped_df_test

41
['LGESP' 'LGEUS' 'LGEGF' 'LGEDG' 'LGEEG' 'LGEFS' 'LGEIN' 'LGEIL' 'LGEIS'
 'LGEAR' 'LGETK' 'LGESJ' 'LGEPH' 'LGEPR' 'LGEVH' 'LGEAP' 'LGECL' 'LGECB'
 'LGEHK' 'LGEMS' 'LGETH' 'LGEMK' 'LGEBN' 'LGEPS' 'LGEUK' 'LGESA' 'LGESL'
 'LGEYK' 'LGEAG' 'LGEEF' 'LGECH' 'LGECI' 'LGEAF' 'LGEPL' 'LGEES' 'LGEHS'
 'LGEAS' 'LGELF' 'LGERO' 'LGEPT' 'LGEML']


,response_corporate,is_converted
34,LGESP,1045
38,LGEUS,1009
18,LGEIL,907
25,LGEPH,289
27,LGEPR,212
24,LGEMS,181
15,LGEGF,151
6,LGECB,126
20,LGEIS,123
9,LGECL,110


In [305]:
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Park Yoonseo\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to C:\Users\Park
[nltk_data]     Yoonseo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Park
[nltk_data]     Yoonseo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [306]:
# 토큰화 & 불용어 제거
from nltk.corpus import stopwords
from konlpy.tag import Okt
from nltk.stem import PorterStemmer

# 불용어 제거를 위한 영어 불용어 리스트
stop_words = set(stopwords.words('english'))

# 어간 추출
stemmer = PorterStemmer()

# 텍스트 데이터
text = df['expected_timeline'].astype(str)

# 각 문장을 단어로 토큰화한 후 불용어를 제거하는 과정
tokenized_sentences = []
for sentence in text:
    # 문장을 단어로 토큰화
    tokens = word_tokenize(sentence)
    # 불용어 제거 & 어간 추출
    filtered_sentence = [stemmer.stem(word.lower()) for word in tokens if word.lower() not in stop_words]
    tokenized_sentences.append(filtered_sentence)

tokenized_sentences = pd.Series(tokenized_sentences)
df['expected_timeline'] = tokenized_sentences

In [307]:
# 토큰화 & 불용어 제거
from nltk.corpus import stopwords
from konlpy.tag import Okt
from nltk.stem import PorterStemmer

# 불용어 제거를 위한 영어 불용어 리스트
stop_words = set(stopwords.words('english'))

# 어간 추출
stemmer = PorterStemmer()

# 텍스트 데이터
text = df_test['expected_timeline'].astype(str)

# 각 문장을 단어로 토큰화한 후 불용어를 제거하는 과정
tokenized_sentences = []
for sentence in text:
    # 문장을 단어로 토큰화
    tokens = word_tokenize(sentence)
    # 불용어 제거 & 어간 추출
    filtered_sentence = [stemmer.stem(word.lower()) for word in tokens if word.lower() not in stop_words]
    tokenized_sentences.append(filtered_sentence)

tokenized_sentences = pd.Series(tokenized_sentences)
df_test['expected_timeline'] = tokenized_sentences

In [308]:
# 정수 인코딩
from sklearn.preprocessing import LabelEncoder

# 모든 리스트의 요소를 하나의 리스트로 펼치기
all_values = [val for sublist in tokenized_sentences for val in sublist]

# 정수 인코딩을 위해 LabelEncoder 사용
label_encoder = LabelEncoder()
label_encoder.fit(all_values)

# 각 리스트의 요소를 정수로 변환하여 저장
df['expected_timeline'] = tokenized_sentences.apply(lambda x: [label_encoder.transform([item])[0] for item in x])

df['expected_timeline']
# 이거를 어떻게 이용해야 하니..

0              [6]
1              [6]
2        [4, 1, 5]
3              [7]
4        [4, 1, 5]
           ...    
59294          NaN
59295          NaN
59296          NaN
59297          NaN
59298          NaN
Name: expected_timeline, Length: 59299, dtype: object

In [309]:
# 정수 인코딩
from sklearn.preprocessing import LabelEncoder

# 모든 리스트의 요소를 하나의 리스트로 펼치기
all_values = [val for sublist in tokenized_sentences for val in sublist]

# 정수 인코딩을 위해 LabelEncoder 사용
label_encoder = LabelEncoder()
label_encoder.fit(all_values)

# 각 리스트의 요소를 정수로 변환하여 저장
df_test['expected_timeline'] = tokenized_sentences.apply(lambda x: [label_encoder.transform([item])[0] for item in x])

df_test['expected_timeline']
# 이거를 어떻게 이용해야 하니..

0             [6]
1             [6]
2       [4, 1, 5]
3             [7]
4       [4, 1, 5]
          ...    
5266          [6]
5267          [6]
5268    [4, 1, 5]
5269          [6]
5270          [6]
Name: expected_timeline, Length: 5271, dtype: object

In [310]:
print(df['business_area'].nunique())
print(df['business_area'].unique())

12
['corporate / office' nan 'education' 'hotel & accommodation'
 'hospital & health care' 'special purpose' 'residential (home)'
 'government department' 'retail' 'factory'
 'power plant / renewable energy' 'transportation' 'public facility']


In [311]:
def label_encoding(series: pd.Series) -> pd.Series:
    """범주형 데이터를 시리즈 형태로 받아 숫자형 데이터로 변환합니다."""

    my_dict = {}

    # 모든 요소를 문자열로 변환
    series = series.astype(str)

    for idx, value in enumerate(sorted(series.unique())):
        my_dict[value] = idx
    series = series.map(my_dict)

    return series

In [312]:
label_columns = [
    "customer_country",
    # "business_subarea",
    "business_area",
    "business_unit",
    "customer_type",
    "enterprise",
    "customer_job",
    "inquiry_type",
    "product_category",
    # "product_subcategory",
    # "product_modelname",
    # "customer_country.1",
    "customer_position",
    "response_corporate",
    "expected_timeline",
]

df_all = pd.concat([df[label_columns], df_test[label_columns]])

for col in label_columns:
    df_all[col] = label_encoding(df_all[col])

In [313]:
for col in label_columns:  
    df[col] = df_all.iloc[: len(df)][col]
    df_test[col] = df_all.iloc[len(df) :][col]

In [314]:
x_train, x_val, y_train, y_val = train_test_split(
    df.drop("is_converted", axis=1),
    df["is_converted"],
    test_size=0.2,
    shuffle=True,
    random_state=400,
)

In [315]:
feature_at_index_13 = df.iloc[:, ]
saudi_arabia_indices = feature_at_index_13[feature_at_index_13 == "Saudi Arabia'"].index
print("Indices where 'Saudi Arabia' occurs:", saudi_arabia_indices)

Indices where 'Saudi Arabia' occurs: RangeIndex(start=0, stop=59299, step=1)


In [316]:
print(feature_at_index_13.shape)

(59299, 23)


In [317]:
import catboost
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from catboost import CatBoostClassifier, Pool, cv
import os
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
from matplotlib import pyplot as plt
from sklearn.metrics import accuracy_score
import optuna
from optuna import Trial
from optuna.samplers import TPESampler
from sklearn.metrics import log_loss
from optuna import Trial
from optuna.samplers import TPESampler
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import KFold

In [318]:
cat_cols = df.columns[df.dtypes == 'object'].to_list() # 카테고리 칼럼
num_cols = df.columns[df.dtypes != 'object'].to_list() # 카테고리 칼럼이 아닌 수

In [319]:
print(len(cat_cols)+len(num_cols))

23


In [320]:
df.shape

(59299, 23)

In [321]:
df_test.shape

(5271, 21)

In [322]:
model = CatBoostClassifier()

In [323]:
model = DecisionTreeClassifier()

In [324]:
model.fit(x_train.fillna(0), y_train)

DecisionTreeClassifier()

In [325]:
def get_clf_eval(y_test, y_pred=None):
    confusion = confusion_matrix(y_test, y_pred, labels=[True, False])
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, labels=[True, False])
    recall = recall_score(y_test, y_pred)
    F1 = f1_score(y_test, y_pred, labels=[True, False])

    print("오차행렬:\n", confusion)
    print("\n정확도: {:.4f}".format(accuracy))
    print("정밀도: {:.4f}".format(precision))
    print("재현율: {:.4f}".format(recall))
    print("F1: {:.4f}".format(F1))

In [326]:
pred = model.predict(x_val.fillna(0))
get_clf_eval(y_val, pred)

오차행렬:
 [[  526   421]
 [  455 10458]]

정확도: 0.9261
정밀도: 0.5362
재현율: 0.5554
F1: 0.5456


In [327]:
# 예측에 필요한 데이터 분리
x_test = df_test.drop(["is_converted", "id"], axis=1)

In [328]:
test_pred = model.predict(x_test.fillna(0))
sum(test_pred) # True로 예측된 개수

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- business_subarea
- customer_country.1
- customer_idx
- lead_owner
- product_modelname
- ...
Feature names seen at fit time, yet now missing:
- bant_submit
- com_reg_ver_win_rate
- historical_existing_cnt
- id_strategic_ver
- it_strategic_ver
- ...


In [ ]:
# 제출 데이터 읽어오기 (df_test는 전처리된 데이터가 저장됨)
df_sub = pd.read_csv("submission.csv")
df_sub["is_converted"] = test_pred

# 제출 파일 저장
df_sub.to_csv("submission.csv", index=False)